_Camille Avestruz (UChicago)_, _Alex Malz (NYU)_, \& Others

Our understanding of how folks currently use this code is based on the [Analytic Bias Estimate (ABE)](https://github.com/LSSTDESC/clmassmod/blob/master/notebooks/Analytic_bias_estimate.ipynb) and [Statistical Power Check (SPC)](https://github.com/LSSTDESC/clmassmod/blob/master/notebooks/Statistical%20Power%20Check.ipynb) demos.

This is a wishlist for how we'd like to use `clmassmod`.  We will refactor the code to make this notebook run!

In [ ]:
import clmassmod as clmm

Assumptions about different input model information are specified as properties of a `Prior` object.  Some of these are already objects: 

* a cosmology (`colossus.cosmology.cosmology`)
* a shear profile model (`colossus.halo.profile_dk14` or `colossus.halo.profile_nfw`)
* a mass-concentration relation model (`colossus.halo.concentration.concentration`)
* center offset


assumptions as separate objects with their own parameters
* `m200`
* `zcluster`
* `radii` (radial range)
* `zsource`

Maybe we can use named tuples for these? or properties of Prior, so Prior.cosmology

In [ ]:
assumption1 = 
assumption2 = 

In [ ]:
assumptions = clmm.Prior(assumption1, assumption2)

Now we have enough information to make a `CLMassMod` object, the black box that will do all the work.

In [ ]:
massmod = CLMassMod(assumptions)

Similarly, the data can be a `Data` object with certain properties.  In the previous version, these properties include:

* `m200`
* `zcluster`
* `radii` (radial range)
* `zsource`
* field of view
* noise properties

galaxypicker?



In [ ]:
dataproperty1 = 
dataproperty2 = 

In [ ]:
data = clmm.Data(dataproperty1, datproperty2)

There are some functions that preprocess the data in various ways:

* `colossus.halo.profile_dk14.surfaceDensity` or `colossus.halo.profile_nfw.surfaceDensity` operating on `radii`
* `colossus.halo.profile_dk14..deltaSigma` or `colossus.halo.profile_nfw.deltaSigma` operating on `radii`

These functions will be rolled into `clmm.CLMassMod.preprocess`

In [ ]:
massmod.preprocess(data)

We want to perform operations with the `CLMassMod` object.

`model` is a keyword for the type of model (e.g. Diemer, NFW) taking `model_params` as a dict of parameters.

* `model=stack`
* `model=prob`

`Fitter` class



In [ ]:
bin_definitions = range(begin, end, number_of_bins)

In [ ]:
bin_assignments, pdfs, mles = massmod.individual_profile_fits(model_name, in_data, calc_params, bin_definitions)
    # parallelization option in calc_params
    # writes one file per halo

need a `massmod.read` and `massmod.write`

In [ ]:
 = massmod.ensemble_fit(bin_definitions, directory_name, method_name)
    # does the sampling as method_name kw
    # opens every file in directory

In [ ]:
likelihoods, posteriors, mass_dist_bias_samps, mass_dist_scatter_samps = massmod.compare(truth)

In [ ]:
massmod.sample(subset_of_data, parameter_to_sample, sampling_method, method_parameters)

In [ ]:
fit_info = massmod.fit_shear(model_name, model_params)

In [ ]:
massmod.plot_shear()

In [ ]:
profiles = blackbox.fits()

In [ ]:
inputs = profiles

In [ ]:
truth = clmm.Truth(otherdata

In [ ]:
bias_meas = blackbox.final(truth)

In [ ]:
outputs = bias_meas